In [ ]:
import pickle
import xarray as xa
from scipy.stats import pearsonr
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='DejaVu Serif') 
# mpl.rc('font', serif='Helvetica Neue') 

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_mswep_month = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_mswep = pickle.load(pfile)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-False-month-True-unforced-False-joint-False', 'rb') as pfile:
    metrics_gpcp_month = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-False-month-False-unforced-False-joint-False', 'rb') as pfile:
    metrics_gpcp = pickle.load(pfile)

In [ ]:
pcmdi = xa.open_dataset('/p/lustre3/shiduan/sst/PCMDI/tos_input4MIPs_SSTsAndSeaIce_CMIP_PCMDI-AMIP-1-1-8_gn_195001-202112_2p5x2p5_masked.nc')
pcmdi = pcmdi["tos"]
pcmdi = pcmdi.sel(time=slice('1983-01-01', '2020-12-31'))

In [ ]:
tos_nino34 = pcmdi.sel(lat=slice(-5, 5), lon=slice(190, 240))
tos_nino34['time'] = metrics_mswep['obs_pc']['time']

In [ ]:
gb = tos_nino34.groupby('time.month')
tos_nino34_anom = gb - gb.mean(dim='time')
index_nino34 = tos_nino34_anom.mean(dim=['lat', 'lon'])

In [ ]:
(-index_nino34).plot()
metrics_mswep['obs_pc'].plot(label='MSWEP-AllM')
metrics_mswep_month['obs_pc'].plot(label='MSWEP-MbyM')
metrics_gpcp['obs_pc'].plot(label='GPCP-AllM')
metrics_gpcp_month['obs_pc'].plot(label='GPCP-MbyM')
plt.legend()
print(pearsonr(index_nino34, metrics_mswep['obs_pc']))
print(pearsonr(index_nino34, metrics_mswep_month['obs_pc']))
print(pearsonr(index_nino34, metrics_gpcp['obs_pc']))
print(pearsonr(index_nino34, metrics_gpcp_month['obs_pc']))

In [ ]:
from matplotlib.gridspec import GridSpec

In [ ]:
fig = plt.figure(figsize=(9, 6))
grid = GridSpec(ncols=2, nrows=2, figure=fig)
# ax = fig.add_subplot(121)
ax = fig.add_subplot(grid[0, 0])
r, p = pearsonr(index_nino34, metrics_mswep['obs_pc'])
ax.scatter(index_nino34, metrics_mswep['obs_pc'], marker='o',  
           color='red', label='AllM r='+str(round(r, 2)))
r, p = pearsonr(index_nino34, metrics_mswep_month['obs_pc'])
ax.scatter(index_nino34, metrics_mswep_month['obs_pc'], marker='o', facecolor='none', 
           edgecolor='black', label='MbyM r='+str(round(r, 2)))
ax.legend()
ax.set_xlabel('Niño 3.4 Index')
ax.set_title('MSWEP')
# ax = fig.add_subplot(122)
ax = fig.add_subplot(grid[0, 1])
r, p = pearsonr(index_nino34, metrics_gpcp['obs_pc'])
ax.scatter(index_nino34, metrics_gpcp['obs_pc'], marker='s', 
           color='red', label='AllM r='+str(round(r, 2)))
r, p = pearsonr(index_nino34, metrics_gpcp_month['obs_pc'])
ax.scatter(index_nino34, metrics_gpcp_month['obs_pc'], marker='s', facecolor='none', 
           edgecolor='black', label='MbyM r='+str(round(r, 2)))
ax.legend()
ax.set_xlabel('Niño 3.4 Index')
ax.set_title('GPCP')
ax = fig.add_subplot(grid[1, :])
ax.plot(np.arange(456), metrics_gpcp['obs_pc'], label='GPCP')
ax.plot(np.arange(456), metrics_mswep['obs_pc'], label='MSWEP')
ax.plot(np.arange(456), -index_nino34, color='black', linewidth=2, label='Niño 3.4 (reversed)')
ax.set_xticks(np.arange(10, 456, 100))
time = metrics_gpcp['obs_pc'].time.data
label = [str(time[i])[:7] for i in range(10, 456, 100)]
ax.set_xticklabels(label)
ax.legend()
fig.supylabel('Pseudo-PC', x=0, y=.7)
plt.tight_layout()
plt.savefig('enso-pr.png', dpi=180, bbox_inches='tight')
plt.show()